In [2]:
import re
import pandas as pd
import spacy
import spacy_fastlang  # can fix model issue
from bs4 import BeautifulSoup
from spacy_cleaner import processing, Cleaner

In [ ]:
df = pd.read_csv('YOUR TWITTER DATA PATH', encoding='unicode_escape')
df = df.loc[df['keys'] == '#CircularEconomy']
print('orginal data shape ', df.shape)
df.drop_duplicates(subset=['full_text'], inplace=True)
print('first remove duplicates ', df.shape)
data = [str(row.full_text) for index, row in df.iterrows()]
pattern = r'RT @\w+: '
data = [text for text in data if not re.search(pattern, text)]
print('second remove duplicates ', len(data))
data = list(set(data))
print('third remove duplicates ', len(data))

In [37]:
''' preprocess '''
cleaned_data = []
model = spacy.load("en_core_web_sm")
cleaner = Cleaner(
    model,
    processing.remove_stopword_token,
    processing.remove_punctuation_token,
    processing.remove_email_token,
    processing.remove_url_token,
    processing.mutate_lemma_token,
)

for html_text in df['full_text']:
    soup = BeautifulSoup(html_text, 'html.parser')
    soup_text = soup.get_text().lower()
    cleaned_data.append(soup_text)

print('spaCy preprocess start!')
cleaned_data = cleaner.clean(cleaned_data)
print(len(cleaned_data))

model.add_pipe("language_detector")
english_cleaned_data = []
english_created_at = []
for i, doc in enumerate(cleaned_data):
    # checking "rt @"
    if not doc.startswith("rt @"):
        tmp = model(doc)
        if tmp._.language == 'en' and tmp._.language_score >= 0.5:
            english_cleaned_data.append(doc)
            english_created_at.append(str(df['created_at'].iloc[i]).split()[0])

print('spaCy preprocess done!')
print('Number of English tweets: ', len(english_cleaned_data))

english_cleaned_data = [re.sub(r"@[\w-]+", "", text).strip() for text in english_cleaned_data]
df_clean = pd.DataFrame({'full_text': english_cleaned_data, 'created_at': english_created_at})
df_clean.to_csv('/content/test.csv', index=False)

<ipython-input-37-2b0ac92eadae>:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_text, 'html.parser')
<ipython-input-37-2b0ac92eadae>:14: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(html_text, 'html.parser')


spaCy preprocess start!


Cleaning Progress: 100%|██████████| 7282/7282 [00:37<00:00, 193.65it/s]


7282
spaCy preprocess done!
Number of English tweets:  3922


In [39]:
df_clean.head(10)

,full_text,created_at
0,understand true close net zero small thing pop...,2023-02-08
1,sponsorship opp available remade circularecono...,2023-02-08
2,juniper j4350 jb service router 4350 series ju...,2023-02-08
3,download onro app today mobile device start se...,2023-02-08
4,belkin omniview 8 port kvm switch dual console...,2023-02-08
5,phew today launch ¨c count ® em action plan se...,2023-02-08
6,vintage video game noob mean 1up goodwillfind ...,2023-02-08
7,visualize mismanage plastic waste country v/ ...,2023-02-08
8,true art collector masterpiece world famous li...,2023-02-08
9,circulareconomy planet friendly way turn unwan...,2023-02-08
